<a href="https://colab.research.google.com/github/oublalkhalid/MoroccoAI-Data-Challenge/blob/main/training/Detect_Morocco_Plate_Licence_(Part_1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DETECT MOROCCO PLATES LICENSES (Deep Neural Network Part I)

* Our goal here is to train a Yolo v3 and a poly Yolo in its tiny version, I have already trained this one but the performance is not appreciated due to the weak image with annotation.

* I specify here, that I am interested in the whole plate the recognition of the numbers or the alphabet is not treated in this level. In addition, we will save the weights of our network at the output of this notebook. Also we will point out that a large part of the data has been generated to solve the problem of Imbalanced data and that a log transformation has been done to answer the issue of Imbalanced data.

> **Note:**
A large part of this work was done under an AI station with an RTX Geforce 3090 card. However, to facilitate the visualization of this approach, we made this notebook under Colab Pro (you can also run this code under Kaggle, or airflow).



# **Getting started and uploading data and creating a sym link**

Please provide what you are using as a machine_run to upload the files to your notebook.

In [ ]:
machine_run="Colab" #local # kaggle

if machine_run=="Colab":
  %cd ..
  from google.colab import drive
  drive.mount('/content/gdrive')
  !ln -s /content/gdrive/My\ Drive/ /mydrive
  !ls /mydrive

if  machine_run=="kaggle":
    pass

if  machine_run=="local":
 path="" # Please upload our main folder containing all necessary data (moroccoAI/data/..)

/
Mounted at /content/gdrive
 Assessment_OUBLAL-Khalid.ipynb
'Ballon Project - Documents'
'Centrale Lyon 2020'
'Colab Notebooks'
 cora
 Cours_space_mechanics
 data
 Data-IA
 Detector
 gain
 Image_TP4
 INF554
 INF554-khalid
'ISAE SUPAERO 2021'
 khalid_me.png
 khalid.png
'Les gain'
'Les Relevés'
'M2 TSI'
'Ma famille '
'ML Lectures_polytechnique'
 MoroccoAI-challenge-OUBLAL-2021
'My Drive'
"Programmation X-l'ORIENTation"
'Python projet'
 Readme.md
 ResNet32.pth
 Resnet_Pretrained18.pth
 ResNet.pth
 Soutenance_OUBLAL-Khalid-2021-Septembre.pdf
'Tactics for TOEIC'
'TP2 corrélation'
 TP_Bruit
 TP_Capteur
 TP-OMR
'TP space thermal - Model using SYSTEMA Software'
 Untitled0.ipynb
 VGGNet.pth
 vgg_pred.pth
 vlcsnap-7084-07-07-11h09m08s377.png
 vlcsnap-7952-05-12-10h20m40s308.png


#**Cloning and Building Darknet**
Here I use only the Darknet library to train the YOLOv3 model with the custom data we obtained in part 0. Please downloading the library from pjreddie/darknet's repository, and adjusting Makefile to enable OPENCV and GPU for darknet and then building the darknet.

In [ ]:
# cloning darknet repo
!git clone https://github.com/pjreddie/darknet.git

fatal: destination path 'darknet' already exists and is not an empty directory.


In [ ]:
# changing the makefile to have GPU and OpneCv enabled
%cd darknet/
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/darknet


In [ ]:
#Uninstall the current CUDA version
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

#Download CUDA 10.0
!wget --no-clobber https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
#install CUDA kit dpkg
!dpkg -i cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
!sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
!apt-get update
!apt-get install cuda-10-0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-fs-prebuilt' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'


In [ ]:
# Making sure to use GPU with CUDA
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [ ]:
# Building the darknet
!make

mkdir -p obj
mkdir -p backup
mkdir -p results
gcc -Iinclude/ -Isrc/ -DOPENCV `pkg-config --cflags opencv`  -DGPU -I/usr/local/cuda/include/ -DCUDNN  -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -c ./src/gemm.c -o obj/gemm.o
./src/gemm.c: In function ‘time_gpu’:
./src/gemm.c:232:9: warning: ‘cudaThreadSynchronize’ is deprecated [-Wdeprecated-declarations]
         cudaThreadSynchronize();
         ^~~~~~~~~~~~~~~~~~~~~
In file included from /usr/local/cuda/include/cuda_runtime.h:96:0,
                 from include/darknet.h:11,
                 from ./src/utils.h:5,
                 from ./src/gemm.c:2:
/usr/local/cuda/include/cuda_runtime_api.h:947:57: note: declared here
 extern __CUDA_DEPRECATED __host__ cudaError_t CUDARTAPI cudaThreadSynchronize(void);
                                                         ^~~~~~~~~~~~~~~~~~~~~
gcc -Iinclude/ -Isrc/ -DOPENCV `pkg-config --cflags opencv`  -DGPU -I/usr/local/cuda/include/ -DCUDNN 

### **Create & Copy MoroccoAI-challenge-OUBLAL to your Colab env**


In [ ]:
# This has all the images data that we use to train and test
!cp /mydrive/MoroccoAI-challenge-OUBLAL-2021/datasets/dataset_plates.zip ../

### **Extracting zip to Colab VM under darket/custom folder**

In [ ]:
!unzip ../dataset.zip -d ../darknet/custom

Archive:  ../dataset.zip
replace ../darknet/custom/dataset/train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ../darknet/custom/dataset/train.csv  
  inflating: ../darknet/custom/dataset/licence.names  
  inflating: ../darknet/custom/dataset/image/450.jpg  
  inflating: ../darknet/custom/dataset/image/0.jpg  
  inflating: ../darknet/custom/dataset/image/1.jpg  
  inflating: ../darknet/custom/dataset/image/10.jpg  
  inflating: ../darknet/custom/dataset/image/100.jpg  
  inflating: ../darknet/custom/dataset/image/101.jpg  
  inflating: ../darknet/custom/dataset/image/102.jpg  
  inflating: ../darknet/custom/dataset/image/103.jpg  
  inflating: ../darknet/custom/dataset/image/104.jpg  
  inflating: ../darknet/custom/dataset/image/105.jpg  
  inflating: ../darknet/custom/dataset/image/106.jpg  
  inflating: ../darknet/custom/dataset/image/107.jpg  
  inflating: ../darknet/custom/dataset/image/108.jpg  
  inflating: ../darknet/custom/dataset/image/109.jpg  
  inflating: ../dar



# **Train the YOLOv3 model using the Darknet**

Darknet provides YOLOv3 model support that we used to train with our custom data. We are using `darknet53.conv.74` default weights to begin our training (transfer learning), and a configuration file `yolov3_plates.cfg` that supports out neural network layers. Upon trianing the model, we will have our custom weights that will be used to predict the license plates. 

Coping Default weights and custom config for Object detection from GDrive

### Coping Default weights and custom config for Object detection from GDrive

In [ ]:
plates_yolo_config = "/content/gdrive/MyDrive/MoroccoAI-challenge-OUBLAL-2021/models/yolov3-license-plates.cfg"
plates_yolo_weights = "/content/gdrive/MyDrive/MoroccoAI-challenge-OUBLAL-2021/models/yolov3_plates_final.weights"
plates_classes = ['Plate']
chars_classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a','b','d', 'h', 'j', 'm', 'p', 'waw', 'ww']

chars_yolo_config = "/content/gdrive/MyDrive/MoroccoAI-challenge-OUBLAL-2021/models/yolov3_character.cfg"
chars_yolo_weights = "/content/gdrive/MyDrive/MoroccoAI-challenge-OUBLAL-2021/models/yolov3_chars_final.weights"

In [ ]:
!cp /content/gdrive/MyDrive/MoroccoAI-challenge-OUBLAL-2021/models/yolov3-license-plates.cfg ../darknet/

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-12-22 17:05:58--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.1’

darknet53.conv.74.1 100%[===================>] 154.96M  20.7MB/s    in 8.2s    

2021-12-22 17:06:07 (18.8 MB/s) - ‘darknet53.conv.74.1’ saved [162482580/162482580]



In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-12-22 09:49:24--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.5’

darknet53.conv.74.5 100%[===================>] 154.96M   105MB/s    in 1.5s    

2021-12-22 09:49:26 (105 MB/s) - ‘darknet53.conv.74.5’ saved [162482580/162482580]

--2021-12-22 09:49:26--  http://../darknet/
Resolving .. (..)... failed: Name or service not known.
wget: unable to resolve host address ‘..’
FINISHED --2021-12-22 09:49:26--
Total wall clock time: 1.5s
Downloaded: 1 files, 155M in 1.5s (105 MB/s)


In [ ]:
!cp /mydrive/MoroccoAI-challenge-OUBLAL-2021/train.txt  ../darknet/custom/dataset/
!cp /mydrive/MoroccoAI-challenge-OUBLAL-2021/test.txt  ../darknet/custom/dataset/

In [ ]:
!cp /mydrive/MoroccoAI-challenge-OUBLAL-2021/yolov3-license-plates.cfg /content/darknet/cfg

cp: cannot stat '/mydrive/MoroccoAI-challenge-OUBLAL-2021/yolov3-license-plates.cfg': No such file or directory


# **Training the YOLOv3 model with CustomData**

In [ ]:
pwd

'/darknet'

In [ ]:
!./darknet detector train custom/dataset/licence.data yolov3-license-plates.cfg darknet53.conv.74 -dont_show

yolov3-license-plates
layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   608 x 608 x   3   ->   608 x 608 x  32  0.639 BFLOPs
    1 conv     64  3 x 3 / 2   608 x 608 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    2 conv     32  1 x 1 / 1   304 x 304 x  64   ->   304 x 304 x  32  0.379 BFLOPs
    3 conv     64  3 x 3 / 1   304 x 304 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    4 res    1                 304 x 304 x  64   ->   304 x 304 x  64
    5 conv    128  3 x 3 / 2   304 x 304 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    6 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
    7 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    8 res    5                 152 x 152 x 128   ->   152 x 152 x 128
    9 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
   10 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
   11 res    8 

# **Image Utils**

In [ ]:
import cv2
import matplotlib.pyplot as plt
from google.colab import files

def upload():
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

def download(path):
  from google.colab import files
  files.download(path)

# def dipslayImage(path):
#   %matplotlib inline
#   image = cv2.imread(path)
#   height, width = image.shape[:2]
#   resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)
#   fig = plt.gcf()
#   fig.set_size_inches(18, 10)
#   plt.axis("off")
#   plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
#   plt.show()


Upload Image

In [ ]:
# Uploading image to root directory
%cd ../darknet
upload()

/darknet


Saving m3_1.jpg to m3_1.jpg
saved file m3_1.jpg


Predicted Image

In [ ]:
!cd .../
!ls

/bin/bash: line 0: cd: .../: No such file or directory
backup		   include	 LICENSE.mit  README.md
cfg		   LICENSE	 LICENSE.v1   results
custom		   LICENSE.fuck  m3_1.jpg     scripts
darknet53.conv.74  LICENSE.gen	 Makefile     src
data		   LICENSE.gpl	 obj	      yolov3-license-plates.cfg
examples	   LICENSE.meta  python


In [ ]:
# Prediction
from IPython.display import Image
%cd ../darknet
!./darknet detector test custom/dataset/license_plate.data yolov3_plates.cfg custom/models/yolov3_plates_final.weights m3_0.jpg
Image('predictions.jpg')